In [299]:
!pip install polars numpy implicit tqdm pyarrow


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [300]:
!pip install pandas==1.3.1


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [301]:
import polars as pl
import numpy as np
from tqdm import tqdm
from scipy.sparse import csr_matrix
import implicit
import pandas as pd
RANDOM_STATE = 42
N_PREDICTIONS = 100
from pathlib import Path
data_path = Path('data')
train_logs = data_path / 'hh_recsys_train_hh.pq'
test_logs = data_path / 'hh_recsys_test_hh.pq'
vacancies_content = data_path / 'hh_recsys_vacancies.pq'
submission_sample = data_path / 'hh_recsys_sample.pq'

In [302]:
#pl.read_parquet(str(submission_sample))

In [337]:
train = pl.read_parquet(str(train_logs))
test = pl.read_parquet(str(test_logs))



##### Контент вакансий
До их разбора пока руки не дошли

In [304]:
#vacancies = pl.read_parquet(str(vacancies_content))
#vacancies

In [338]:
train = train.to_pandas()
test = test.to_pandas()
# vacancies = vacancies.to_pandas()

### EDA (комментарии в процессе)

In [306]:
train

,user_id,session_id,vacancy_id,action_type,action_dt
0,u_332060,s_28301374,"[v_2571684, v_488179, v_2389179, v_1393783, v_...","[2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, ...","[2023-11-01T00:40:58.105000000, 2023-11-01T00:..."
1,u_1057881,s_33868982,[v_665861],[2],[2023-11-01T00:23:51.452000000]
2,u_1036784,s_32474802,[v_2594840],[2],[2023-11-01T00:52:34.023000000]
3,u_786220,s_14060785,"[v_1473781, v_1622905, v_1621959, v_2289180, v...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 2, ...","[2023-11-01T00:58:20.793000000, 2023-11-01T01:..."
4,u_639152,s_23205986,"[v_695738, v_22433, v_1590524, v_502496, v_200...","[2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, ...","[2023-11-01T01:14:20.828000000, 2023-11-01T00:..."
...,...,...,...,...,...
3463059,u_202578,s_8306993,[v_1499243],[2],[2023-11-13T21:01:41.813000000]
3463060,u_895531,s_7293998,[v_2041381],[2],[2023-11-14T17:51:22.169000000]
3463061,u_317562,s_9866576,[v_503192],[2],[2023-11-14T20:03:54.988000000]
3463062,u_225581,s_19724962,[v_2526106],[2],[2023-11-14T20:52:51.985000000]


#### Временные рамки лога, частота событий в день

In [257]:
train.sample(frac=0.05).action_dt.apply(lambda x: pd.to_datetime(x[0]).date()).value_counts().sort_index()

2023-11-01    15424
2023-11-02    15233
2023-11-03    12142
2023-11-04     6303
2023-11-05     5986
2023-11-06     8732
2023-11-07    16782
2023-11-08    16359
2023-11-09    15660
2023-11-10    13421
2023-11-11     6475
2023-11-12     7137
2023-11-13    16928
2023-11-14    16571
Name: action_dt, dtype: int64

In [258]:
test.sample(frac=0.5).action_dt.apply(lambda x: pd.to_datetime(x[0]).date()).value_counts().sort_index()

2023-11-15    6620
2023-11-16    6906
2023-11-17    5821
2023-11-18    2827
2023-11-19    3137
2023-11-20    8145
2023-11-21    8138
Name: action_dt, dtype: int64

#### Распределение юзеров по кол-ву сессий

In [11]:
train.user_id.value_counts()

count    882409.000000
mean          3.924557
std           5.444649
min           1.000000
25%           1.000000
50%           2.000000
75%           4.000000
max         828.000000
Name: user_id, dtype: float64

In [12]:
test.user_id.value_counts().describe()

count    83189.0
mean         1.0
std          0.0
min          1.0
25%          1.0
50%          1.0
75%          1.0
max          1.0
Name: user_id, dtype: float64

#### Распределение вакансий по частоте

In [25]:
train_vacancy_counts = train.explode('vacancy_id').vacancy_id.value_counts()
train_vacancy_counts.describe()

count    1.458353e+06
mean     1.475368e+01
std      5.082756e+01
min      1.000000e+00
25%      1.000000e+00
50%      4.000000e+00
75%      1.200000e+01
max      1.325400e+04
Name: vacancy_id, dtype: float64

In [17]:
train.explode('vacancy_id').user_id.value_counts().describe()

count    882409.00000
mean         24.38333
std         289.57993
min           1.00000
25%           2.00000
50%           7.00000
75%          22.00000
max      260689.00000
Name: user_id, dtype: float64

#### Распределение юзеров по кол-ву просмотренных вакансий

In [21]:
def get_user_vacancy_count(df):
    return df.explode('vacancy_id').user_id.value_counts()
train_user_vacancy_count = get_user_vacancy_count(train)
test_user_vacancy_count = get_user_vacancy_count(test)
train_user_vacancy_count.describe()

count    882409.00000
mean         24.38333
std         289.57993
min           1.00000
25%           2.00000
50%           7.00000
75%          22.00000
max      260689.00000
Name: user_id, dtype: float64

### Препроцессинг

сделал более мудреный препроц на классах, чтобы можно было одинаково обрабатывать train и тест и чтобы препроц был сразу из коробки обратимым - сначала нужно идшники перегнать из строк в инты для производительности, а для сабмита вернуть их к исходному виду.

In [357]:

class IdParser:
    def __init__(self, original_col, parsed_col, prefix):
        self.original_col = original_col
        self.parsed_col = parsed_col
        self.prefix = prefix
    def transform(self, df):
        df[self.original_col] = df[self.original_col].apply(self.id_parser)
        return df
    def inverse_transform(self, df):
        df[self.original_col] = df[self.original_col].apply(self.id_construct)
        return df
    def id_parser(self, user_id_str):
        return int(user_id_str[len(self.prefix):])
    def id_construct(self, id_):
        return f'{self.prefix}{id_}'

class IdArrayParser(IdParser):
    def id_parser(self, id_arr):
        return [super(IdArrayParser, self).id_parser(id_) for id_ in id_arr] 

    def id_construct(self, id_arr):
        return [super(IdArrayParser, self).id_construct(id_) for id_ in id_arr] 


class Pipeline:
    def __init__(self, stages):
        self.stages = stages
    def transform(self, x):
        for stage in self.stages:
            x = stage.transform(x)
        return x
    def inverse_transform(self, x):
        for stage in reversed(self.stages):
            x = stage.inverse_transform(x)
        return x
class ColumnRename:
    def __init__(self, col_mapping, skip_reverse=True):
        self.mapping = col_mapping
        self.skip_reverse = skip_reverse
        self.inverse_mapping = dict(((v,k) for k,v in col_mapping.items()))
    def transform(self, df):
        return df.drop(self.mapping.values(), axis=1).rename(self.mapping, axis=1)
    def inverse_transform(self, df):
        existing_keys = {k:v for k,v in self.inverse_mapping.items() if k in df.columns}
        return df.drop(self.inverse_mapping.values(), axis=1).rename(existing_keys, axis=1)
class SessionDt:
    def __init__(self, dt_arr_col):
        self.dt_col = dt_arr_col
    def transform(self, df):
        df['session_dt'] = df[self.dt_col].apply(lambda dts: dts[0])
        return df
    def inverse_transform(self, df):
        return df
als_preproc_pipeline = Pipeline([
    IdParser('user_id', 'user_id_int', 'u_'),
    IdArrayParser('vacancy_id', 'vacancy_arr', 'v_'), 
    SessionDt('action_dt'),
    #ColumnRename({'user_id_int':'user_id', 'vacancy_arr':'vacancy_id'})
])

def prepare_validation(history):
    # оставить юзеров, у которых был последним отклик
    history_for_val = history[history.action_type.apply(lambda x: x[-1] == 1 and len(x) > 1)]
    history_for_val['next_item'] = history_for_val.vacancy_id.apply(lambda x: x[-1])
    history_for_val['vacancy_id'] = history_for_val.vacancy_id.apply(lambda x: x[:-1])
    history_for_val['action_type'] = history_for_val.action_type.apply(lambda x: x[:-1])
    return history_for_val
    


In [364]:
train_val_split_date = '2023-11-14'
als_train = als_preproc_pipeline.transform(train.copy())
# train/val сплит пока не готов
als_val = prepare_validation(als_train[als_train.session_dt >= train_val_split_date])
als_train = als_train[als_train.session_dt < train_val_split_date]
als_test = als_preproc_pipeline.transform(test.copy())

<ipython-input-357-189b88fad050>:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history_for_val['next_item'] = history_for_val.vacancy_id.apply(lambda x: x[-1])
<ipython-input-357-189b88fad050>:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history_for_val['vacancy_id'] = history_for_val.vacancy_id.apply(lambda x: x[:-1])
<ipython-input-357-189b88fad050>:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [367]:
als_train

,user_id,session_id,vacancy_id,action_type,action_dt,session_dt
0,332060,s_28301374,"[2571684, 488179, 2389179, 1393783, 2608935, 1...","[2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, ...","[2023-11-01T00:40:58.105000000, 2023-11-01T00:...",2023-11-01 00:40:58.105
1,1057881,s_33868982,[665861],[2],[2023-11-01T00:23:51.452000000],2023-11-01 00:23:51.452
2,1036784,s_32474802,[2594840],[2],[2023-11-01T00:52:34.023000000],2023-11-01 00:52:34.023
3,786220,s_14060785,"[1473781, 1622905, 1621959, 2289180, 1473781, ...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 2, ...","[2023-11-01T00:58:20.793000000, 2023-11-01T01:...",2023-11-01 00:58:20.793
4,639152,s_23205986,"[695738, 22433, 1590524, 502496, 2001124, 1740...","[2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, ...","[2023-11-01T01:14:20.828000000, 2023-11-01T00:...",2023-11-01 01:14:20.828
...,...,...,...,...,...,...
3463055,489124,s_32651757,[1975545],[2],[2023-11-13T13:54:31.150000000],2023-11-13 13:54:31.150
3463056,415800,s_25725261,[303627],[2],[2023-11-13T17:41:06.362000000],2023-11-13 17:41:06.362
3463057,54897,s_7396605,[2669097],[2],[2023-11-13T19:11:40.615000000],2023-11-13 19:11:40.615
3463058,999352,s_21061102,[499781],[2],[2023-11-13T19:17:43.379000000],2023-11-13 19:17:43.379


In [254]:
als_preproc_pipeline.inverse_transform(als_train)

,user_id,session_id,vacancy_id,action_type,action_dt,session_dt
0,u_332060,s_28301374,"[v_2571684, v_488179, v_2389179, v_1393783, v_...","[2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, ...","[2023-11-01T00:40:58.105000000, 2023-11-01T00:...",2023-11-01 00:40:58.105
1,u_1057881,s_33868982,[v_665861],[2],[2023-11-01T00:23:51.452000000],2023-11-01 00:23:51.452
2,u_1036784,s_32474802,[v_2594840],[2],[2023-11-01T00:52:34.023000000],2023-11-01 00:52:34.023
3,u_786220,s_14060785,"[v_1473781, v_1622905, v_1621959, v_2289180, v...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 2, ...","[2023-11-01T00:58:20.793000000, 2023-11-01T01:...",2023-11-01 00:58:20.793
4,u_639152,s_23205986,"[v_695738, v_22433, v_1590524, v_502496, v_200...","[2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, ...","[2023-11-01T01:14:20.828000000, 2023-11-01T00:...",2023-11-01 01:14:20.828
...,...,...,...,...,...,...
3463059,u_202578,s_8306993,[v_1499243],[2],[2023-11-13T21:01:41.813000000],2023-11-13 21:01:41.813
3463060,u_895531,s_7293998,[v_2041381],[2],[2023-11-14T17:51:22.169000000],2023-11-14 17:51:22.169
3463061,u_317562,s_9866576,[v_503192],[2],[2023-11-14T20:03:54.988000000],2023-11-14 20:03:54.988
3463062,u_225581,s_19724962,[v_2526106],[2],[2023-11-14T20:52:51.985000000],2023-11-14 20:52:51.985


In [255]:
als_test

,user_id,session_id,vacancy_id,action_type,action_dt,session_dt
0,482520,s_25018731,"[2597196, 1223061, 1223061]","[2, 2, 1]","[2023-11-19T12:03:13.089000000, 2023-11-19T12:...",2023-11-19 12:03:13.089
1,582132,s_481216,"[470400, 470400, 1530783]","[2, 1, 2]","[2023-11-21T15:39:47.981000000, 2023-11-21T15:...",2023-11-21 15:39:47.981
2,212584,s_16918781,"[1572055, 1572055, 1572055, 953153, 953153, 14...","[2, 3, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1]","[2023-11-16T08:41:47.031000000, 2023-11-16T08:...",2023-11-16 08:41:47.031
3,425177,s_17505104,"[1375331, 1922852]","[2, 2]","[2023-11-17T12:42:18.513000000, 2023-11-17T12:...",2023-11-17 12:42:18.513
4,700997,s_15528830,"[2152997, 2152997, 1217630]","[2, 1, 2]","[2023-11-16T17:22:53.530000000, 2023-11-16T17:...",2023-11-16 17:22:53.530
...,...,...,...,...,...,...
83184,499368,s_19158589,"[1716634, 2232327, 2232327, 2232327, 408610, 2...","[2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2]","[2023-11-18T03:15:15.530000000, 2023-11-18T03:...",2023-11-18 03:15:15.530
83185,1084868,s_24756162,"[395706, 2405231]","[2, 2]","[2023-11-15T18:06:16.550000000, 2023-11-15T18:...",2023-11-15 18:06:16.550
83186,1138032,s_25291467,"[1213925, 1213925, 2359832, 2359832]","[2, 1, 2, 1]","[2023-11-19T23:19:44.092000000, 2023-11-19T23:...",2023-11-19 23:19:44.092
83187,608102,s_18813020,"[933773, 933773, 2021127, 1508520, 2021127, 20...","[3, 2, 2, 2, 2, 1, 2]","[2023-11-16T15:40:09.408000000, 2023-11-16T15:...",2023-11-16 15:40:09.408


# Simple ALS

In [368]:
from abc import ABC, abstractmethod
from dataclasses import dataclass

from sklearn.preprocessing import LabelEncoder


# Пока эти классы используются чисто как документация, чтобы было понятно с чем работает модель
@dataclass
class UserItems:
    user_id: int
    item: list

@dataclass
class UserItemHistory(UserItems):
    action_type: list
    session_dt: list

@dataclass
class Predict(UserItems):
    score: list

# Все что нужно АЛС модели от датасета
class UsrHistory(ABC):
    @abstractmethod
    def get_unique_users(self):
        pass
    @abstractmethod
    def get_unique_items(self):
        pass
    @abstractmethod
    def get_per_item_interactions(self):
        pass

@dataclass
class SessionTrainPoint(UserItemHistory):
    user_id: int
    seen_items: List[UserItemHistory]
    actions: List[int]
    next_items: List[int]
    recoms: List[int]

# обертка над пандасовским датасетом, чтобы проще писать рек модель
class PandasUsrHistory(UsrHistory):
    def __init__(self, data):
        self.data = data
    def get_unique_users(self):
        return self.data['user_id'].unique()
    def get_unique_items(self):
        return self.data['vacancy_id'].explode().unique()
    def get_per_item_interactions(self):
        return self.data[['user_id', 'vacancy_id', 'action_type']] \
        .rename({'vacancy_id':'item'}, axis=1) \
        .explode(['item', 'action_type'])

# интерфейс к мапперу действия юзера в вес АЛС
class ActionWeightMapper(ABC):
    @abstractmethod
    def transform(self, weight):
        pass
# маппер на основе словаря
class PandasDictWeightMapper(ActionWeightMapper):
    def __init__(self, mapping):
        self.action_weights = mapping
    def transform(self, actions):
        return actions.apply(lambda action: self.action_weights[action])

# класс, который берет на себя ремап из хешей int64 в последовательные идшники 
# и собирает на них датасет для матричной факторизации
# так же умеет инференсить рекомы для юзера из исходного датасета
class AlsDataset:
    def __init__(self, mf_model, action_weights_mapper):
        self.mf_model = mf_model
        self.action_weights = action_weights_mapper
    
    def prepare_dataset(self, user_histories: UsrHistory):
        #pairs = df[['user_id', 'item', 'action_type']].explode(['item', 'action_type'])
        self.user_encoder = LabelEncoder().fit(user_histories.get_unique_users())
        self.item_encoder = LabelEncoder().fit(user_histories.get_unique_items())
        
        pairs = user_histories.get_per_item_interactions()
        users = self.user_encoder.transform(pairs['user_id'])
        vacancies = self.item_encoder.transform(pairs['item'])
        preferences = self.action_weights.transform(pairs['action_type'])
        uv_mat = csr_matrix((preferences, (users, vacancies)))
        self.uv_mat = uv_mat
        return uv_mat
    
    def recommend(self, user, topk=5):
        cuser = self.user_encoder.transform([user])
        # почему-то items, scores распаковываются от als_model в массив в массиве
        # пока просто живу с этим
        items, scores = self.mf_model.recommend(cuser, self.uv_mat[cuser], N=topk)
        items = self.item_encoder.inverse_transform(items[0])
        return Predict(user, items, scores[0])
    





In [437]:

als_model = implicit.als.AlternatingLeastSquares(
    factors=150,
    random_state=RANDOM_STATE,
    iterations=25,
    calculate_training_loss=True,
    regularization=0.001
)
   

als_model.fit(als_uv_mat_train)

  0%|          | 0/25 [00:00<?, ?it/s]

In [439]:
als_dataset_pair_gen = AlsDataset(
    als_model, 
    PandasDictWeightMapper({
            1: 4.0,
            2: 1.0,
            3: 2.0
        })
)

als_train_maped = PandasUsrHistory(als_train)

als_uv_mat_train = als_dataset_pair_gen.prepare_dataset(als_train_maped)#.transform(als_train)
#als_model.fit(als_uv_mat_train)

In [442]:
als_dataset_pair_gen.recommend(332060,100)

Predict(user_id=332060, item=array([2182, 2212918, 357578, 955457, 2486828, 853222, 373282, 998961,
       362970, 1388809, 2213400, 2027660, 564001, 2023918, 843713,
       2561089, 1714969, 471884, 1211665, 868388, 104548, 2730601,
       1441263, 270515, 1646383, 2321392, 2614873, 2450251, 2508104,
       484383, 391249, 1251511, 1141859, 391164, 2069610, 2031240,
       1330862, 1765666, 171916, 1728188, 2125195, 599759, 1338815, 40511,
       304860, 2008724, 774525, 2397386, 128273, 2415073, 720393, 2644159,
       848636, 2103741, 1746866, 1011908, 1486343, 1485671, 1381426,
       2556604, 342234, 416538, 250065, 1324926, 2675525, 1737040, 339887,
       2307509, 1592451, 566480, 1544265, 583062, 403277, 2099821,
       1602597, 600126, 2566060, 463281, 2255345, 617994, 1776961,
       1184813, 335213, 2193979, 943748, 1939010, 1268739, 2340962,
       1405073, 1540167, 231511, 1425457, 1721632, 1020908, 44197, 46353,
       1699482, 325721, 198869, 1077964], dtype=object), sco

In [66]:
list(als_dataset_pair_gen.user2idx.keys())[:10]

[332060,
 1057881,
 1036784,
 786220,
 639152,
 456512,
 584261,
 733524,
 1114237,
 407319]

In [20]:
als_model.user_factors.shape, als_model.item_factors.shape

((882409, 150), (1458353, 150))

In [458]:
train_users = als_train_maped.get_unique_users()

def predict_for_dataset(test, train_users):
    predictions = []
    test_users = test['user_id'].to_list()
    test_vacancies = test['vacancy_id']
    for user, vacs in tqdm(zip(test_users, test_vacancies), total=len(test_users)):
        if user not in train_users:
            items = vacs
            scores = [0 for i in vacs]
            predict = Predict(user, items, scores)
        else:        
            predict = als_dataset_pair_gen.recommend(user, 100)
        predictions.append(predict)

    predicts = pd.DataFrame(predictions, index=test.index)
    return predicts

In [466]:
def merge_predictions(dataset, predictions):
    predictions = predictions.rename({'vacancy_id':'predictions'}, axis=1)
    dataset = dataset.merge(predictions['user_id predictions'.split()],
                left_index=True, right_index=True) 
    return dataset
def mrr_rank_query(true_item, predict_items):
    rank = 1
    found = False
    for predict in predict_items:
        if predict == true_item:
            found = True
            break
        rank += 1
    if found :
        return 1 / rank
    else:
        return 0
def mrr_rank_dataset(dataset):
    return dataset.apply(lambda row: mrr_rank_query(row.next_item, row.predictions), axis=1).mean()



In [432]:
val_merged

,user_id_x,session_id,vacancy_id,action_type,action_dt,session_dt,next_item,user_id_y,predictions
1354,932816,s_7462242,"[1597209, 1391503, 1924314, 2699408, 1991536, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, ...","[2023-11-14T00:01:12.906000000, 2023-11-13T23:...",2023-11-14 00:01:12.906,1372635,932816,"[1597209, 1391503, 1924314, 2699408, 1991536, ..."
1371,1096623,s_32400309,"[448896, 2723503, 2590661, 183233, 306809, 290...","[2, 2, 2, 1, 2, 2, 2, 2, 2]","[2023-11-14T06:48:42.973000000, 2023-11-14T06:...",2023-11-14 06:48:42.973,2723503,1096623,"[239331, 117522, 21191, 44039, 62864]"
1378,245980,s_1462983,"[1855648, 425220, 824985, 2712817, 1847402, 17...","[1, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 1, ...","[2023-11-14T08:10:30.306000000, 2023-11-14T08:...",2023-11-14 08:10:30.306,1729514,245980,"[17473, 3423, 44169, 14979, 14980]"
1395,353039,s_4206845,"[1351391, 1108173, 1407081, 1407081]","[1, 1, 2, 1]","[2023-11-14T10:22:06.905000000, 2023-11-14T10:...",2023-11-14 10:22:06.905,1712504,353039,"[12184, 30743, 1793, 30303, 7037]"
1411,147741,s_13100528,"[2136050, 1181724, 2244554, 856021, 2072570, 2...","[1, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, ...","[2023-11-14T11:49:40.150000000, 2023-11-14T10:...",2023-11-14 11:49:40.150,2083700,147741,"[9082, 5243, 239331, 8521, 16428]"
...,...,...,...,...,...,...,...,...,...
7186,637605,s_6406369,[1159774],[2],"[2023-11-14T17:46:06.759000000, 2023-11-14T17:...",2023-11-14 17:46:06.759,1159774,637605,"[67617, 48066, 8675, 54148, 139813]"
7190,102063,s_16401250,"[514464, 2114181, 953153, 2114181, 1773749, 27...","[1, 1, 1, 2, 1, 1]","[2023-11-14T19:07:14.669000000, 2023-11-14T19:...",2023-11-14 19:07:14.669,753987,102063,"[76581, 106887, 124374, 87441, 63285]"
7191,901395,s_16431660,"[1129019, 1129019]","[2, 2]","[2023-11-14T19:09:58.154000000, 2023-11-14T19:...",2023-11-14 19:09:58.154,1129019,901395,"[122, 105235, 9258, 1793, 171090]"
7852,106429,s_4743320,[1765702],[2],"[2023-11-14T06:00:24.718000000, 2023-11-14T06:...",2023-11-14 06:00:24.718,1765702,106429,"[55752, 68192, 68191, 68188, 82867]"


In [447]:
test_predictions

,user_id,item,score
0,482520,"[2650515, 367301, 2710612, 1911300, 1768044, 1...","[0.0966707, 0.09484507, 0.09027968, 0.08949311..."
1,582132,"[1347362, 1798396, 2586135, 1617380, 1566952, ...","[0.034510054, 0.03395419, 0.033202425, 0.03304..."
2,212584,"[1572055, 1572055, 1572055, 953153, 953153, 14...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,425177,"[2650515, 367301, 2710612, 1911300, 2145060, 1...","[0.0055885236, 0.0055036065, 0.005219233, 0.00..."
4,700997,"[1431178, 2395102, 1480870, 1161228, 1643983, ...","[0.33196074, 0.3161099, 0.30549717, 0.23109135..."
...,...,...,...
95,201306,"[1315032, 1247743, 1169994, 449237, 534342, 53...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
96,323258,"[171057, 2509509, 125394, 755644, 1287262, 133...","[0.03815136, 0.02765447, 0.025019072, 0.022970..."
97,696520,"[639254, 1050982, 431781, 2116102, 1347290, 61...","[0.119163215, 0.10739049, 0.10440427, 0.098625..."
98,210178,"[2093855, 2093855]","[0, 0]"


In [457]:
test_predictions

,user_id,item,score
0,482520,"[2650515, 367301, 2710612, 1911300, 1768044, 1...","[0.0966707, 0.09484507, 0.09027968, 0.08949311..."
1,582132,"[1347362, 1798396, 2586135, 1617380, 1566952, ...","[0.034510054, 0.03395419, 0.033202425, 0.03304..."
2,212584,"[1572055, 1572055, 1572055, 953153, 953153, 14...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,425177,"[2650515, 367301, 2710612, 1911300, 2145060, 1...","[0.0055885236, 0.0055036065, 0.005219233, 0.00..."
4,700997,"[1431178, 2395102, 1480870, 1161228, 1643983, ...","[0.33196074, 0.3161099, 0.30549717, 0.23109135..."
5,687201,"[1465910, 1586766, 2667504, 1726234, 1973551, ...","[0.41027686, 0.40593547, 0.39603016, 0.3834262..."
6,573219,"[701974, 2692350, 1879498, 670383, 1081658, 23...","[0.3273009, 0.29532242, 0.28517905, 0.28038108..."
7,1082782,"[460169, 964765, 2337447, 2730052, 1431178, 14...","[0.073951125, 0.05781182, 0.04164842, 0.035235..."
8,81444,"[497984, 1024479, 507080, 2691293, 2302139, 11...","[1.3593356e-06, 9.896954e-07, 9.788534e-07, 7...."
9,858357,"[2322256, 2730052, 1124183, 366963, 1481785, 1...","[0.009810198, 0.007663134, 0.007218748, 0.0071..."


In [435]:
val_predictions = predict_for_dataset(als_val, train_users)
val_merged = merge_predictions(als_val, val_predictions)
mrr_rank_dataset(val_merged)

0.11439939717644977

In [470]:
test_predictions

,user_id_int,vacancy_arr,vacancy_id,user_id
0,482520,"[2650515, 367301, 2710612, 1911300, 1768044, 1...","[v_2650515, v_367301, v_2710612, v_1911300, v_...",u_482520
1,582132,"[1347362, 1798396, 2586135, 1617380, 1566952, ...","[v_1347362, v_1798396, v_2586135, v_1617380, v...",u_582132
2,212584,"[1572055, 1572055, 1572055, 953153, 953153, 14...","[v_1572055, v_1572055, v_1572055, v_953153, v_...",u_212584
3,425177,"[2650515, 367301, 2710612, 1911300, 2145060, 1...","[v_2650515, v_367301, v_2710612, v_1911300, v_...",u_425177
4,700997,"[1431178, 2395102, 1480870, 1161228, 1643983, ...","[v_1431178, v_2395102, v_1480870, v_1161228, v...",u_700997
5,687201,"[1465910, 1586766, 2667504, 1726234, 1973551, ...","[v_1465910, v_1586766, v_2667504, v_1726234, v...",u_687201
6,573219,"[701974, 2692350, 1879498, 670383, 1081658, 23...","[v_701974, v_2692350, v_1879498, v_670383, v_1...",u_573219
7,1082782,"[460169, 964765, 2337447, 2730052, 1431178, 14...","[v_460169, v_964765, v_2337447, v_2730052, v_1...",u_1082782
8,81444,"[497984, 1024479, 507080, 2691293, 2302139, 11...","[v_497984, v_1024479, v_507080, v_2691293, v_2...",u_81444
9,858357,"[2322256, 2730052, 1124183, 366963, 1481785, 1...","[v_2322256, v_2730052, v_1124183, v_366963, v_...",u_858357


In [ ]:

test_predictions = predict_for_dataset(als_test, train_users)
test_predictions = als_dataset_pipeline.inverse_transform(test_predictions.rename({'item':'vacancy_id'},axis=1))
submission = merge_predictions(als_test, test_predictions)
submission


 11%|█         | 9112/83189 [10:44<1:16:04, 16.23it/s]

In [170]:
submission = merge_predictions(test, predictions)
pl.from_pandas(submission).select(['user_id', 'session_id', 'predictions']).write_parquet('als_submission.pq')